# Importando base de dados Sinesp VDE 2023-2025 para PostgreSQL

Vamos trabalhar com as bases de dados do Validador de Dados Estatísticos do Sistema Nacional de Informações de Segurança Pública (Sinesp-VDE). Os dados têm um formato consolidado desde 2023, então vamos trabalhar com esses arquivos.

In [1]:
years = [2023, 2024, 2025]

## Importação

In [2]:
from pathlib import Path

import pandas as pd
from pandas import DataFrame as DF

from codetiming import Timer

def load_spreadsheet_and_cache_csv(spreadsheet_filename, sheet_name=0, header=0, suffix=''):
    filename = spreadsheet_filename.replace('.xlsx', '').replace('.ods', '')
    csv_filename = Path(f'csvcache/{filename}{suffix}.csv')
    if csv_filename.exists():
        print(f'{spreadsheet_filename} disponíveis em formato CSV, carregando de {csv_filename}')
        with Timer(f'Carga de {csv_filename}'):
            df = pd.read_csv(csv_filename, low_memory=False) # low_memory evita warning the "mixed types"
    else:
        print(f'{spreadsheet_filename}  não disponíveis em formato CSV, carregando de {spreadsheet_filename}')
        with Timer(f'Carga de {spreadsheet_filename}'):
            df = pd.read_excel(spreadsheet_filename, sheet_name=sheet_name, header=header)
        print(f'Salvando {spreadsheet_filename} como CSV (porque CSVs são mais rápidos de carregar)')
        with Timer(f'Salvamento de {csv_filename}'):
            df.to_csv(csv_filename)
        print(f'{spreadsheet_filename} salva como CSV em {csv_filename}')
    return df

dfs = []

for y in years:
    filename = f'BancoVDE {y}.xlsx'
    dfs.append(load_spreadsheet_and_cache_csv(filename))

BancoVDE 2023.xlsx disponíveis em formato CSV, carregando de csvcache/BancoVDE 2023.csv
Elapsed time: 1.0429 seconds
BancoVDE 2024.xlsx disponíveis em formato CSV, carregando de csvcache/BancoVDE 2024.csv
Elapsed time: 1.0204 seconds
BancoVDE 2025.xlsx disponíveis em formato CSV, carregando de csvcache/BancoVDE 2025.csv
Elapsed time: 0.8145 seconds


Todos os dataframes têm as mesmas colunas:

In [3]:
columns = zip(*[df.columns for df in dfs])
DF({
  'year': years  
} | {
    f'c{i}' : [cs[y-years[0]] for y in years] for i, cs in enumerate(columns)
}).transpose()

,0,1,2
year,2023,2024,2025
c0,Unnamed: 0,Unnamed: 0,Unnamed: 0
c1,uf,uf,uf
c2,municipio,municipio,municipio
c3,evento,evento,evento
c4,data_referencia,data_referencia,data_referencia
c5,agente,agente,agente
c6,arma,arma,arma
c7,faixa_etaria,faixa_etaria,faixa_etaria
c8,feminino,feminino,feminino


Vejamos algumas linhas:

In [4]:
dfs[0].head()

,Unnamed: 0,uf,municipio,evento,data_referencia,agente,arma,faixa_etaria,feminino,masculino,nao_informado,total_vitima,total,total_peso,abrangencia,formulario
0,0,AC,NÃO INFORMADO,Apreensão de Cocaína,2023-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.366,Estadual,Formulário 5
1,1,AL,NÃO INFORMADO,Apreensão de Cocaína,2023-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.729,Estadual,Formulário 5
2,2,AM,NÃO INFORMADO,Apreensão de Cocaína,2023-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,294.635,Estadual,Formulário 5
3,3,AP,NÃO INFORMADO,Apreensão de Cocaína,2023-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.530,Estadual,Formulário 5
4,4,BA,NÃO INFORMADO,Apreensão de Cocaína,2023-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.470,Estadual,Formulário 5


Todas têm o mesmo formato, aparentemente. Vamos concatenar os dataframes:

In [5]:
df = pd.concat(dfs)

`Unnamed: 0` é um identificador, não nos interessa muito mas podemos preservar para referenciar de volta:

In [6]:
df.rename(columns={'Unnamed: 0': 'id_sinesp_vde'}, inplace=True)

Valor `total` não está claro, vejamos algumas linhas:

In [7]:
df[~df['total'].isna()].head()

,id_sinesp_vde,uf,municipio,evento,data_referencia,agente,arma,faixa_etaria,feminino,masculino,nao_informado,total_vitima,total,total_peso,abrangencia,formulario
648,648,AC,NÃO INFORMADO,Arma de Fogo Apreendida,2023-01-01,NaN,Carabina,NaN,NaN,NaN,NaN,NaN,1.0,NaN,Estadual,Formulário 6
649,649,AC,NÃO INFORMADO,Arma de Fogo Apreendida,2023-01-01,NaN,Espingarda,NaN,NaN,NaN,NaN,NaN,5.0,NaN,Estadual,Formulário 6
650,650,AC,NÃO INFORMADO,Arma de Fogo Apreendida,2023-01-01,NaN,Fuzil,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Estadual,Formulário 6
651,651,AC,NÃO INFORMADO,Arma de Fogo Apreendida,2023-01-01,NaN,Metralhadora,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Estadual,Formulário 6
652,652,AC,NÃO INFORMADO,Arma de Fogo Apreendida,2023-01-01,NaN,Outra,NaN,NaN,NaN,NaN,NaN,16.0,NaN,Estadual,Formulário 6


Nos casos, se refere a armas de fogo. Haverá outros possíveis tipos de eventos?

In [8]:
df[~df['total'].isna()]['evento'].unique()

array(['Arma de Fogo Apreendida', 'Atendimento pré-hospitalar',
       'Busca e salvamento', 'Combate a incêndios',
       'Emissão de Alvarás de licença', 'Furto de veículo',
       'Mandado de prisão cumprido', 'Realização de vistorias',
       'Roubo a instituição financeira', 'Roubo de carga',
       'Roubo de veículo', 'Tráfico de drogas'], dtype=object)

Valor total refere-se, aparentemente, a contagem de quaisquer "objetos" de relevância para o incidente.'

## Definição de DDL

### Tabela `AgregacaoEvento` (principal)

Esses são os campos, sendo que alguns nós vamos converter em chaves estrangeiras:

```sql
CREATE TABLE AgregacaoEvento (
    id                 SERIAL PRIMARY KEY,
    id_sinesp_vde      INTEGER NOT NULL,
    data_referencia    DATE NOT NULL,
    vitimas_femininas  INTEGER,
    vitimas_masculinas INTEGER,
    vitimas_nao_inform INTEGER,
    total_vitimas      INTEGER,
    total_objetos      INTEGER,
    total_peso         INTEGER,
    tipo_evento_id     INTEGER NOT NULL REFERENCES TipoEvento(id),
    uf_id              INTEGER NOT NULL REFERENCES UF(id),
    municipio_id       INTEGER NOT NULL REFERENCES Municipio(id),
    abrangencia_id     INTEGER NOT NULL REFERENCES Abrangencia(id),
    formulario_id      INTEGER NOT NULL REFERENCES Formulario(id),
    orgao_agente_id    INTEGER REFERENCES OrgaoAgente(id),
    arma_id            INTEGER REFERENCES Arma(id),
    faixa_etaria_id    INTEGER REFERENCES FaixaEtaria(id)
);

```

### Tabela `TipoEvento`

Para essa tabela, precisamos saber o tamanho ideal da coluna com o tipo do evento:

In [9]:
def values_and_sizes(df, coluna):
    valores = df[coluna].unique()
    return DF({
        coluna: valores,
        'tamanho_nome': [len(v) if isinstance(v, str) else 0 for v in valores]
    })
    
eventos_df = values_and_sizes(df, 'evento')
eventos_df

,evento,tamanho_nome
0,Apreensão de Cocaína,20
1,Apreensão de Maconha,20
2,Arma de Fogo Apreendida,23
3,Atendimento pré-hospitalar,26
4,Busca e salvamento,18
5,Combate a incêndios,19
6,Emissão de Alvarás de licença,29
7,Estupro,7
8,Estupro de vulnerável,21
9,Feminicídio,11


Os nomes podem ser bem grandes e descritivos, temos um com 66 letras. Vamos usar 80 então.

O tipo de evento também define se há:
* contagem de vítimas (masculinas, femininas, desconhecidas, total);
* faixa etária
* peso de drogas apreendidas;
* contagem de "objetos" (desde armas apreendidas a mandatos executados);
* tipo de arma;

Isso será útil para categorizar os tipos de eventos, então vou gerar colunas nos eventos com esses dados:

```sql
CREATE TABLE TipoEvento (
    id               SERIAL PRIMARY KEY,
    evento           VARCHAR(80) UNIQUE NOT NULL,
    tem_vitima       BOOLEAN NOT NULL,
    tem_faixa_etaria BOOLEAN NOT NULL,
    tem_arma         BOOLEAN  NOT NULL,
    tem_peso         BOOLEAN NOT NULL,
    tem_objeto       BOOLEAN NOT NULL
);
```

### Tabela `OrgaoAgente`

Vejamos os valores da coluna `agente`:

In [10]:
agentes_df = values_and_sizes(df, 'agente')
agentes_df

,agente,tamanho_nome
0,NaN,0
1,Agente de Trânsito,18
2,Bombeiro Militar,16
3,Guarda Municipal,16
4,Polícia Civil,13
5,Polícia Federal,15
6,Polícia Militar,15
7,Polícia Penal,13
8,Polícia Rodoviária Federal,26
9,Profissionais de Perícia,24


Uma coluna  com 30 caracteres parece suficiente:

```sql
CREATE TABLE OrgaoAgente (
    id    SERIAL PRIMARY KEY,
    orgao VARCHAR(30) UNIQUE NOT NULL
);

```

### Tabela `UF`

São as unidades da federação. Seus valores:

In [11]:
ufs_df = values_and_sizes(df, 'uf')
ufs_df.head(3)

,uf,tamanho_nome
0,AC,2
1,AL,2
2,AM,2


Além da sigla, vou querer também o nome inteiro. Vamos usar essa base de dados encontrada no site do Ministério do Desenvolvimento Social [[blog.mds.gov.br](https://blog.mds.gov.br/redesuas/lista-de-municipios-brasileiros/)] para ver o maior nome de estado:

In [12]:
mds_estados_df = pd.read_csv(
    'Lista_Estados_Brasil_Versao_CSV.csv',
    encoding='ISO-8859-1', sep=';')
mds_estados_df.head()

,IBGE,Estado,UF,Região,Qtd Mun,Sintaxe,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,11,Rondônia,RO,Região Norte,52,11'RO',NaN,NaN,PROCV(A2;'Lista de Estados IBGE'!$A$2:$C$28;2;0)
1,12,Acre,AC,Região Norte,22,12'AC',NaN,NaN,NaN
2,13,Amazonas,AM,Região Norte,62,13'AM',NaN,NaN,NaN
3,14,Roraima,RR,Região Norte,15,14'RR',NaN,NaN,NaN
4,15,Pará,PA,Região Norte,144,15'PA',NaN,NaN,NaN


In [13]:
mds_estados_nomes_df = values_and_sizes(mds_estados_df, 'Estado')
mds_estados_nomes_df

,Estado,tamanho_nome
0,Rondônia,8
1,Acre,4
2,Amazonas,8
3,Roraima,7
4,Pará,4
5,Amapá,5
6,Tocantins,9
7,Maranhão,8
8,Piauí,5
9,Ceará,5


O maior nome tem 19 letras, então vamos pôr 25. Também é interessante usar a região, não havia pensando nisso! Vamos encolher o nome e avaliar

In [14]:
mds_estados_df['Região'] = mds_estados_df['Região'].str.replace('Região', '')
mds_regioes_df = values_and_sizes(mds_estados_df, 'Região')
mds_regioes_df

,Região,tamanho_nome
0,Norte,6
1,Nordeste,9
2,Sudeste,8
3,Sul,4
4,Centro-Oeste,13


```sql
CREATE TABLE Regiao (
    id   SERIAL PRIMARY KEY,
    nome VARCHAR(15) UNIQUE NOT NULL
);

CREATE TABLE UF (
    id        SERIAL PRIMARY KEY,
    sigla     CHAR(2) UNIQUE NOT NULL,
    nome      VARCHAR(25) UNIQUE NOT NULL,
    regiao_id INTEGER NOT NULL REFERENCES Regiao(id)
);

```

### Tabela `Municipio`

Podemos pegar nomes dos municípios todos da base de dados de populações por ano do IBGE [[ibge.gov.br](https://www.ibge.gov.br/estatisticas/sociais/populacao/9103-estimativas-de-populacao.html?edicao=44309)]:

In [15]:
ibge_municipios_df = load_spreadsheet_and_cache_csv('POP2025_20251031.ods', sheet_name='Municípios', header=1)
ibge_municipios_df = ibge_municipios_df[ibge_municipios_df.duplicated('NOME DO MUNICÍPIO')].sort_values(by='NOME DO MUNICÍPIO')

POP2025_20251031.ods  não disponíveis em formato CSV, carregando de POP2025_20251031.ods
Elapsed time: 2.1772 seconds
Salvando POP2025_20251031.ods como CSV (porque CSVs são mais rápidos de carregar)
Elapsed time: 0.0168 seconds
POP2025_20251031.ods salva como CSV em csvcache/POP2025_20251031.csv


In [16]:
values_and_sizes(ibge_municipios_df, 'NOME DO MUNICÍPIO').sort_values(by='tamanho_nome')

,NOME DO MUNICÍPIO,tamanho_nome
232,NaN,0
21,Belém,5
52,Cedro,5
57,Conde,5
82,Irati,5
...,...,...
150,Presidente Juscelino,20
148,Presidente Bernardes,20
27,Bom Jesus do Tocantins,22
192,São Francisco de Paula,22


O maior nome de município tem 23 letras, vamos usar 40:

```sql
CREATE TABLE Municipio (
    id           INTEGER PRIMARY KEY,
    nome         VARCHAR(40) UNIQUE NOT NULL,
    uf_id    INTEGER REFERENCES UF(id)
);

```
### Tabela `arma` 


In [17]:
values_and_sizes(df, 'arma')

,arma,tamanho_nome
0,NaN,0
1,Carabina,8
2,Espingarda,10
3,Fuzil,5
4,Metralhadora,12
5,Outra,5
6,Pistola,7
7,Revolver,8
8,Rifle,5
9,Submetralhadora,15


```sql
CREATE TABLE Arma (
    id   SERIAL PRIMARY KEY,
    nome VARCHAR(20) UNIQUE NOT NULL
);

```

### Coluna `faixa_etaria`



In [18]:
values_and_sizes(df, 'faixa_etaria')

,faixa_etaria,tamanho_nome
0,NaN,0
1,Idade Não Informada,19
2,Maior de Idade,14
3,Menor de Idade,14


Então eis a tabela:

```sql
CREATE TABLE FaixaEtaria (
    id    SERIAL PRIMARY KEY,
    faixa VARCHAR(20) UNIQUE NOT NULL
);


```

### Tabela `Abrangencia`

Abrangência só tem um valor

In [19]:
values_and_sizes(df, 'abrangencia')

,abrangencia,tamanho_nome
0,Estadual,8


Mas vou considerar que pode ter a vir outros (municipal? nacional?) então vou criar a tabela.

```sql
CREATE TABLE Abrangencia (
    id                      SERIAL PRIMARY KEY,
    abrangencia VARCHAR(15) UNIQUE NOT NULL
);

```

### Tabela `Formulario`

Formulários têm um número:

In [20]:
df['formulario'].unique()

array(['Formulário 5', 'Formulário 6', 'Formulário 9', 'Formulário 3',
       'Formulário 1', 'Formulário 4', 'Formulário 8', 'Formulário 2',
       'Formulário 7'], dtype=object)

In [21]:
values_and_sizes(df, 'formulario')

,formulario,tamanho_nome
0,Formulário 5,12
1,Formulário 6,12
2,Formulário 9,12
3,Formulário 3,12
4,Formulário 1,12
5,Formulário 4,12
6,Formulário 8,12
7,Formulário 2,12
8,Formulário 7,12


A tabela então pode ter tanto nome quanto número:

```sql
CREATE TABLE Formulario (
    id     SERIAL PRIMARY KEY,
    nome   VARCHAR(15) UNIQUE NOT NULL,
    numero INTEGER UNIQUE NOT NULL
);

```

## Tabelas de população


Vamos querer ter a população dos estados e dos municípios. Só não colocamos como coluna porque a população varia no tempo. O ideal seria a população por mês, mas só há dados da população por ano ([[ibge.gov.br]](https://www.ibge.gov.br/estatisticas/downloads-estatisticas.html)). Vamos criar as tabelas de associação enriquecidas: 

```sql
CREATE_TABLE PopulacaoUF (
    uf_id     INTEGER UNIQUE NOT NULL, REFERENCES Estado(id),
    ano       INTEGER UNIQUE NOT NULL,
    populacao INTEGER UNIQUE NOT NULL,
    PRIMARY KEY (uf_id, ano)
);

CREATE_TABLE PopulacaoMunicipio (
    municipio_id INTEGER UNIQUE NOT NULL, REFERENCES Estado(id),
    ano          INTEGER UNIQUE NOT NULL,
    populacao    INTEGER UNIQUE NOT NULL,
    PRIMARY KEY (nunicipio_id, ano)
);

```

## Valores assoados a tipos de eventos

Alguns valores só estão presentes em alguns tipos de eventos. Vejamos como eles se relacionam, de modo que ao saber o evento, saibamos quais valores esperar:

In [22]:
victim_columns = [
    'feminino', 'masculino', 'nao_informado', 'total_vitima'
]
event_related_columns = victim_columns + [
    'arma', 'faixa_etaria', 
    'total_peso', 'total'
]

event_column_df = df.groupby('evento')[event_related_columns].agg(
    lambda c: c.notna().any()
)
event_column_df['vitima'] =  event_column_df[victim_columns].any(axis=1)
event_column_df.drop(victim_columns, axis=1, inplace=True)
event_column_df


,arma,faixa_etaria,total_peso,total,vitima
evento,,,,,
Apreensão de Cocaína,False,False,True,False,False
Apreensão de Maconha,False,False,True,False,False
Arma de Fogo Apreendida,True,False,False,True,False
Atendimento pré-hospitalar,False,False,False,True,False
Busca e salvamento,False,False,False,True,False
Combate a incêndios,False,False,False,True,False
Emissão de Alvarás de licença,False,False,False,True,False
Estupro,False,False,False,False,True
Estupro de vulnerável,False,False,False,False,True


## DDL final
Com isso, temos o script DDL abaixo:

```sql
CREATE TABLE TipoEvento (
    id                 SERIAL PRIMARY KEY,
    evento             VARCHAR(80) UNIQUE NOT NULL,
    tem_vitima         BOOLEAN NOT NULL,
    tem_faixa_etaria   BOOLEAN NOT NULL,
    tem_arma           BOOLEAN  NOT NULL,
    tem_peso           BOOLEAN NOT NULL,
    tem_objeto         BOOLEAN NOT NULL
);

CREATE TABLE OrgaoAgente (
    id            SERIAL PRIMARY KEY,
    orgao         VARCHAR(30) UNIQUE NOT NULL
);


CREATE TABLE Regiao (
    id           SERIAL PRIMARY KEY,
    nome         VARCHAR(15)
);

CREATE TABLE UF (
    id           SERIAL PRIMARY KEY,
    sigla        CHAR(2) NOT NULL,
    nome         VARCHAR(25) NOT NULL,
    regiao_id    INTEGER NOT NULL REFERENCES Regiao(id)
);

CREATE TABLE Municipio (
    id           SERIAL PRIMARY KEY,
    nome         VARCHAR(40) NOT NULL,
    uf_id    INTEGER REFERENCES UF(id)
);

CREATE TABLE Abrangencia (
    id       INTEGER PRI
)

CREATE TABLE Arma (
    id           SMALLSERIAL PRIMARY KEY,
    nome         VARCHAR(20)
);

CREATE TABLE AgregacaoEvento (
    id                 SERIAL PRIMARY KEY,
    data_referencia    DATE,
    vitimas_femininas  INTEGER,
    vitimas_masculinas INTEGER,
    vitimas_nao_inform INTEGER,
    total_vitimas      INTEGER,
    total_objetos      INTEGER,
    total_peso         INTEGER,
    tipo_evento_id     INTEGER NOT NULL REFERENCES TipoEvento(id),
    uf_id              INTEGER NOT NULL REFERENCES UF(id),
    municipio_id       INTEGER NOT NULL REFERENCES Municipio(id),
    abrangencia_id     INTEGER NOT NULL REFERENCES Abrangencia(id),
    formulario_id      INTEGER NOT NULL REFERENCES Formulario(id),
    orgao_agente_id    INTEGER REFERENCES OrgaoAgente(id),
    arma_id            INTEGER REFERENCES Arma(id),
    faixa_etaria_id    INTEGER REFERENCES FaixaEtaria(id)
);
```

## Definição de CSVs para DML

Vamos converter todos esses dados para CSV, para importar no Postgres.

### Tabelas simples

Estas tabelas ou não tem chaves estrangeiras, ou têm chavs estrangeiras fáceis de resolver posteriormente, e podem ser extraídas da nossa base de dados principal.

In [26]:
tipo_evento_df = event_column_df.rename({c: f'tem_{c}' for c in event_related_columns+['vitima']})
tipo_evento_df.to_csv('csv_dados/TipoEvento.csv')

orgao_agente_df = agentes_df[agentes_df['agente'].notna()].rename({'agente': 'orgao'}, axis=1)
orgao_agente_df[['orgao']].to_csv('csv_dados/OrgaoAgente.csv')

regioes_df = mds_regioes_df.rename({'Região': 'nome'}, axis=1)
regioes_df[['nome']].to_csv('csv_dados/Regiao.csv')

ufs_df = mds_estados_df.rename({'UF': 'sigla', 'Estado': 'nome', 'Região': 'regiao'}, axis=1)
ufs_df[['sigla', 'nome', 'regiao']].to_csv('csv_dados/UF.csv')

municipios_df = df[
    df['municipio']!= 'NÃO INFORMADO'
][
    ['municipio', 'uf']
].drop_duplicates().rename({
    'municipio': 'nome'
}, axis=1)
municipios_df[['nome', 'uf']].to_csv('csv_dados/Municipio.csv')
# Só tem um valor mesmo
open('csv_dados/Abrangencia.csv', 'w').write('abrangencia\nEstadual') # Só há uma abrangência

armas_df = values_and_sizes(df, 'arma')[['arma']].rename({'arma': 'nome'}, axis=1)
armas_df[armas_df['nome'].notna()].to_csv('csv_dados/Arma.csv')

formularios_df = df[['formulario']].drop_duplicates().sort_values(by='formulario')
formularios_df['numero'] = formularios_df['formulario'].str.split(' ').map(lambda s: int(s[1]))
formularios_df = formularios_df.rename({'formulario': 'nome'}, axis=1)
formularios_df[['nome', 'numero']].to_csv('csv_dados/Formulario.csv')

### Tabelas de população

Essas tabelas são mais complicadas, porque precisamos associar com dados de populações. Vamos usar os dados do IBGE. Temos disponíveis os dados para 2024 [[ibge.gov.br](https://www.ibge.gov.br/estatisticas/sociais/populacao/9103-estimativas-de-populacao.html?edicao=41105)] e 2025 [[ibge.gov.br](https://www.ibge.gov.br/estatisticas/sociais/populacao/9103-estimativas-de-populacao.html?edicao=44309)], mas não para 2023, que queremos investigar.

Como só queremos demonstrar o uso de SQL, vamos utilizar os dados de 2021 no lugar [[ibge.gov.br](https://www.ibge.gov.br/estatisticas/sociais/populacao/9103-estimativas-de-populacao.html?edicao=31451)].

Precisamos das populações dos municípios e dos estados. Comecemos pelos municípios:

#### Municípios

In [27]:
from unidecode import unidecode

def get_populacao(
        filename,
        spreadsheet=0,
        header=1,
        index_columns=['UF', 'NOME DO MUNICÍPIO'],
        population_column='POPULAÇÃO ESTIMADA',
        output_columns=['uf', 'municipio', 'populacao'],
        cleanupstr=False
    ):
    df = load_spreadsheet_and_cache_csv(
        filename,
        sheet_name=spreadsheet,
        header=header,
        suffix=spreadsheet
    )
    df = df[df[population_column].notna()]
    if cleanupstr:
        df[population_column] = (
            df[population_column]
                .str
                .replace(r'\(\d+\)', '', regex=True)
                .replace(r'\D', '', regex=True)
                .astype(int)
        )
    df.rename({
        k: v for k, v in zip(index_columns+[population_column], output_columns)
    }, axis=1, inplace=True)
    return df[output_columns]


ibge_populacao_municipios_2021_df = get_populacao('POP2021_20240624.ods', 'Municípios', cleanupstr=True)
ibge_populacao_municipios_2024_df = get_populacao('POP2024_20241230.ods', 'MUNICÍPIOS')
ibge_populacao_municipios_2025_df = get_populacao('POP2025_20251031.ods', 'Municípios')

POP2021_20240624.ods disponíveis em formato CSV, carregando de csvcache/POP2021_20240624Municípios.csv
Elapsed time: 0.0093 seconds
POP2024_20241230.ods  não disponíveis em formato CSV, carregando de POP2024_20241230.ods
Elapsed time: 2.7915 seconds
Salvando POP2024_20241230.ods como CSV (porque CSVs são mais rápidos de carregar)
Elapsed time: 0.0169 seconds
POP2024_20241230.ods salva como CSV em csvcache/POP2024_20241230MUNICÍPIOS.csv
POP2025_20251031.ods  não disponíveis em formato CSV, carregando de POP2025_20251031.ods
Elapsed time: 3.1040 seconds
Salvando POP2025_20251031.ods como CSV (porque CSVs são mais rápidos de carregar)
Elapsed time: 0.0228 seconds
POP2025_20251031.ods salva como CSV em csvcache/POP2025_20251031Municípios.csv


Vamos simular o ano de 2023 com a média de 2021 e 2024. Não é o melhor dado sintético, mas serve bem ao trabalho:

In [28]:
def populacao_media(df1, df2):
# Simulando população de 2023 com dados de 2021 e 2024 (média)
    mean_df = df1.copy()
    mean_df['populacao'] = (df1['populacao']+df2['populacao']) // 2
    return mean_df

def merge_populacao_anos(pop_dict):
    pop_df = pd.DataFrame()
    for y, p_df in pop_dict.items():
        p_df['ano'] = y
        pop_df = pd.concat([pop_df, p_df])
    return pop_df
    

ibge_populacao_municipios_2023_df = populacao_media(
    ibge_populacao_municipios_2021_df,
    ibge_populacao_municipios_2024_df
)

populacao_municipios_df = merge_populacao_anos({
    2023: ibge_populacao_municipios_2023_df,
    2024: ibge_populacao_municipios_2024_df,
    2025: ibge_populacao_municipios_2025_df,
})

populacao_municipios_df

,uf,municipio,populacao,ano
0,RO,Alta Floresta D'Oeste,22684.0,2023
1,RO,Ariquemes,109860.0,2023
2,RO,Cabixi,5378.0,2023
3,RO,Cacoal,92026.0,2023
4,RO,Cerejeiras,16531.0,2023
...,...,...,...,...
5566,GO,Vianópolis,15644.0,2025
5567,GO,Vicentinópolis,9175.0,2025
5568,GO,Vila Boa,4145.0,2025
5569,GO,Vila Propício,6028.0,2025


Verificando se todos os municípios da nossa base de dados de crimes estão na base do IBGE:

In [29]:
municipios_df = df[df['municipio']!='NÃO INFORMADO'][['uf', 'municipio']].drop_duplicates()
municipios_df

,uf,municipio
5508,AC,ACRELÂNDIA
5509,AC,ASSIS BRASIL
5510,AC,BRASILÉIA
5511,AC,BUJARI
5512,AC,CAPIXABA
...,...,...
11132,TO,TOCANTINÓPOLIS
11133,TO,TUPIRAMA
11134,TO,TUPIRATINS
11135,TO,WANDERLÂNDIA


In [30]:
municipios_df[~municipios_df['municipio'].isin(populacao_municipios_df['municipio'].str.upper())]

,uf,municipio


Considerando que todos os municípios da nossa base de crimes estão na base do IBGE (como esperado), podemos simplesmente converter o nome dos municípios da base do IBGE para maiúsculas de modo a obter uma tabela de populações de municípios:

In [31]:
populacao_municipios_df['municipio'] = populacao_municipios_df['municipio'].str.upper()
populacao_municipios_df[['ano', 'uf', 'municipio', 'populacao']].to_csv('csv_dados/PopulacaoMunicipio.csv')

#### UFs

Agora sigamos os mesmos passos para criar o CSV de populações das UFs. Deve ser até um pouco mais fácil, já que UFs não têm nomes repetidos.

In [33]:
ibge_populacao_ufs_2021_df = get_populacao(
    'POP2021_20240624.ods',
    spreadsheet='BRASIL_E_UFs',
    index_columns=['BRASIL E UNIDADES DA FEDERAÇÃO'],
    output_columns=['uf', 'populacao'],
    cleanupstr=True
)
ibge_populacao_ufs_2024_df = get_populacao(
    'POP2024_20241230.ods',
    spreadsheet='BRASIL_E_UFs',
    index_columns=['BRASIL E UNIDADES DA FEDERAÇÃO'],
    output_columns=['uf', 'populacao']
)

ibge_populacao_ufs_2025_df = get_populacao(
    'POP2025_20251031.ods',
    spreadsheet='BRASIL_E_UFs',
    index_columns=['BRASIL E UNIDADES DA FEDERAÇÃO'],
    output_columns=['uf', 'populacao']
)


ibge_populacao_ufs_2023_df = populacao_media(
    ibge_populacao_ufs_2021_df,
    ibge_populacao_ufs_2024_df
)


populacao_ufs_df = merge_populacao_anos({
    2023: ibge_populacao_ufs_2023_df,
    2024: ibge_populacao_ufs_2024_df,
    2025: ibge_populacao_ufs_2025_df,
})

populacao_ufs_df

POP2021_20240624.ods disponíveis em formato CSV, carregando de csvcache/POP2021_20240624BRASIL_E_UFs.csv
Elapsed time: 0.0032 seconds
POP2024_20241230.ods  não disponíveis em formato CSV, carregando de POP2024_20241230.ods
Elapsed time: 2.1159 seconds
Salvando POP2024_20241230.ods como CSV (porque CSVs são mais rápidos de carregar)
Elapsed time: 0.0009 seconds
POP2024_20241230.ods salva como CSV em csvcache/POP2024_20241230BRASIL_E_UFs.csv
POP2025_20251031.ods  não disponíveis em formato CSV, carregando de POP2025_20251031.ods
Elapsed time: 2.1434 seconds
Salvando POP2025_20251031.ods como CSV (porque CSVs são mais rápidos de carregar)
Elapsed time: 0.0008 seconds
POP2025_20251031.ods salva como CSV em csvcache/POP2025_20251031BRASIL_E_UFs.csv


,uf,populacao,ano
0,Brasil,212950694.0,2023
1,Região Norte,18788153.0,2023
2,Rondônia,1780752.0,2023
3,Acre,893753.0,2023
4,Amazonas,4275602.0,2023
...,...,...,...
28,Centro-Oeste,17238818.0,2025
29,Mato Grosso do Sul,2924631.0,2025
30,Mato Grosso,3893659.0,2025
31,Goiás,7423629.0,2025


Como nossa base de crimes tem a sigla da UF e a base da população tem o nome, temos de, de alguma forma, conectá-las. Vamos usar a base do MDS novamente como referência, primeiro, para remover linhas que não são de estados...

In [34]:
populacao_ufs_df = populacao_ufs_df[populacao_ufs_df['uf'].isin(mds_estados_df['Estado'])]
populacao_ufs_df

,uf,populacao,ano
2,Rondônia,1780752.0,2023
3,Acre,893753.0,2023
4,Amazonas,4275602.0,2023
5,Roraima,684753.0,2023
6,Pará,8720715.0,2023
...,...,...,...
27,Rio Grande do Sul,11233263.0,2025
29,Mato Grosso do Sul,2924631.0,2025
30,Mato Grosso,3893659.0,2025
31,Goiás,7423629.0,2025


...e posteriormente convertendo as UFs de nomes para siglas:

In [35]:
populacao_ufs_siglas_df = populacao_ufs_df.merge(
    mds_estados_df[['Estado', 'UF']],
    left_on='uf',
    right_on='Estado'
)
populacao_ufs_siglas_df

,uf,populacao,ano,Estado,UF
0,Rondônia,1780752.0,2023,Rondônia,RO
1,Acre,893753.0,2023,Acre,AC
2,Amazonas,4275602.0,2023,Amazonas,AM
3,Roraima,684753.0,2023,Roraima,RR
4,Pará,8720715.0,2023,Pará,PA
...,...,...,...,...,...
76,Rio Grande do Sul,11233263.0,2025,Rio Grande do Sul,RS
77,Mato Grosso do Sul,2924631.0,2025,Mato Grosso do Sul,MS
78,Mato Grosso,3893659.0,2025,Mato Grosso,MT
79,Goiás,7423629.0,2025,Goiás,GO


In [36]:
populacao_ufs_df = populacao_ufs_siglas_df.drop('uf', axis=1)
populacao_ufs_df = populacao_ufs_df.rename({'UF': 'uf'}, axis=1)
populacao_ufs_df[['ano', 'uf', 'populacao']].to_csv('csv_dados/PopulacaoUF.csv')

### Agregado de occorências

Com valores para todas as outras tabelas, podemos criar um CSV para o agregado. Vamos converter os municípios não informados em null

In [37]:
import numpy as np

agregado_evento_df = df.copy().replace({'NÃO INFORMADO': np.nan})
agregado_evento_df.to_csv('AgregacaoEvento.csv')

## E agora?

Vamos tentar importar esses CSVs!